In [11]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_event_data(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


#### User variables:

In [2]:
output_loc = '../data/'

# Event's list for update:

#### Get last update:

In [3]:
files = get_content(directory=output_loc)
last_file_update = None

for file in files:
    if 'ufc_fighters_stats' in file:
        last_file_update = get_last_update(file_path = output_loc + file)

print('Last update: ' + last_file_update)

Last update: 08-04-2023


#### Get links of events to add to data:

In [4]:
events = get_links(url='http://ufcstats.com/statistics/events/completed?page=all')
events.date = pd.to_datetime(events.date, format='d%-%m-%Y')
events.sort_values(by='date', ascending=False)

# filter this df to only add new matches
if last_file_update == None:
        events_for_update = events.event_url
else:
    df = events[events.date > last_file_update].reset_index(drop=True)
    events_for_update = df.event_url
    
print('Events required update: %d' %len(events_for_update))   
df.tail(3)

Events required update: 2


,date,event_url
0,2023-11-03,http://ufcstats.com/event-details/1ccff7f0cfdf...
1,2023-11-02,http://ufcstats.com/event-details/d26394fc0e8e...


#### Saving file:

In [5]:
df.to_csv(output_loc + 'update_list_events.csv', index=0)
df.tail(3)

,date,event_url
0,2023-11-03,http://ufcstats.com/event-details/1ccff7f0cfdf...
1,2023-11-02,http://ufcstats.com/event-details/d26394fc0e8e...


# Fighter's list for update:

#### Get latest events:

In [6]:
df = pd.read_csv(output_loc + 'update_list_events.csv')
df.tail(3)

,date,event_url
0,2023-11-03,http://ufcstats.com/event-details/1ccff7f0cfdf...
1,2023-11-02,http://ufcstats.com/event-details/d26394fc0e8e...


#### Get list of fighters from even's links:

In [7]:
parts = []

for url in df.event_url:
    parts.extend(get_participants(event_link=url))
    
data = pd.DataFrame(data=parts, columns=['fighter','fighter_url'])
print('Fighters required update: %d' %len(data))
data.tail(3)

Fighters required update: 50


,fighter,fighter_url
47,Denis Tiuliulin,http://ufcstats.com/fighter-details/0112352cb3...
48,Tatsuro Taira,http://ufcstats.com/fighter-details/4461d7e473...
49,Jesus Aguilar,http://ufcstats.com/fighter-details/c051c2e12d...


#### Saving file:

In [8]:
data.to_csv(output_loc + 'update_list_fighters.csv', index=0)
data.tail(3)

,fighter,fighter_url
47,Denis Tiuliulin,http://ufcstats.com/fighter-details/0112352cb3...
48,Tatsuro Taira,http://ufcstats.com/fighter-details/4461d7e473...
49,Jesus Aguilar,http://ufcstats.com/fighter-details/c051c2e12d...
